# Calculate synthetic arrival times using a Python script

`Cake` allows to calculate the arrival times of differen seismic phases ([example here](https://pyrocko.org/docs/current/apps/cake/manual.html#calculate-p-phase-arrivals)). In this task we will compute P and S wave arrivals for the station `GE.KTHA` and the 2020 M<sub>w</sub> 7.0 Samos earthquake and compare them to the picks, we made in the previous module.

We start by importing the different modules from Pyrocko, we need:

In [2]:
from pyrocko import cake, marker, model
import numpy as num

In the next step both the event and station data need to be loaded. We make use of `load_one_event` and `load_stations` for this purpose.

In [ ]:
# Loading of events
events = model.load_one_event(filename='processed_events_pyrocko.yaml')

# Loading of stations
stations = model.load_stations(filename='processed_stations_pyrocko.yaml')

In the next step we need to select the station `GE.KTHA`:

In [4]:
station = None

# Iterate through stations and check for matching network-station-location code
for sta in stations:
    if sta.nsl() == ('GE', 'KTHA', ''):
        station = sta
        
        break

and calculate the distance to the earthquake:

In [5]:
distance = event.distance_to(station) * cake.m2d  # Convert meter to degrees for usage in cake

NameError: name 'event' is not defined

Great, finally we can calculate the arrival times for the different phases based on **cake**. First the synthetic earth model is loaded:

In [10]:
# Load builtin 'prem-no-ocean' model (medium resolution)
earth_model = cake.load_model('prem-no-ocean.m')

We also need to define the phases, we want to extract the travel time for:

In [11]:
# Define the phase to use.
phases = [
    cake.PhaseDef('P'),
    cake.PhaseDef('S')]

Now arrivals are calulated and we directly feed it into `Snuffler` markers for usage in `Snuffler`. 

In [12]:
markers = []

# calculate distances and arrivals and store them as marker:
for arrival in model.arrivals(distance, phases=phases, zstart=event.depth):
    markers.append(marker.PhaseMarker(
        tmin=arrival.t + event.time,
        tmax=arrival.t + event.time,
        phasename=arrival.used_phase,))

NameError: name 'distance' is not defined

In the last step we repeat the loading of waveforms into `Snuffler` (check module 3 - Introduction to seismological toolboxes).

In [16]:
# Import
from pyrocko.squirrel import Squirrel
from pyrocko.gui import snuffler

# Loading of waveforms
sq = Squirrel()
sq.add('data/')

# snuffle
snuffler.snuffle(sq.get_waveforms(), marker=markers)

selecting files... done. 2 files selected.
Looking at files [----------------------------------------] 100% Time:  0:00:00
[Errno 2] No such file or directory: '/home/malde/src/seiscl/cake/-f'
Scanning files [                                          ] N/A% ETA:  --:--:--Cannot read file '/home/malde/.local/share/jupyter/runtime/kernel-2c4ce1e7-6419-4fc7-8076-add2f8439f55.json': No SEED data detected (file: /home/malde/.local/share/jupyter/runtime/kernel-2c4ce1e7-6419-4fc7-8076-add2f8439f55.json)
Scanning files [------------------------------------------] 100% Time:  0:00:00
The following files caused problems and will be ignored:
/home/malde/src/seiscl/cake/-f
/home/malde/.local/share/jupyter/runtime/kernel-2c4ce1e7-6419-4fc7-8076-add2f8439f55.json

 /home/malde/.snufflings/okada/libokada.so: undefined symbol: okada_get_consts
--> run 'make' in okada snuffling directory <--


Finally, compare your picks with the one computed by **cake**. Are they in good agreement? Why could b